In [98]:
import os
import base64
import hashlib
import json
import sys
import urllib
import glob
import pydicom
import pydicom_seg
import tempfile
import shutil
import numpy as np
import pandas as pd
import SimpleITK as sitk 
import subprocess as sp
import pyorthanc

from typing import Literal
from httpx import HTTPError
from datetime import datetime
from pyorthanc import Orthanc
from collections import defaultdict

In [131]:
target_dir_root = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-new-2/*/*segmentation-label.seg.nrrd"

In [64]:
len(glob.glob(target_dir_root))

364

In [65]:
os.path.dirname(glob.glob(target_dir_root)[0])

'/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-new-2/4c177202-954661e4-997f7da5-dd28eff8-83f62076'

In [178]:
files = os.listdir(os.path.dirname(glob.glob(target_dir_root)[0]))

In [179]:
def get_available_dataset(target_dir_root):
    
    cases = glob.glob(target_dir_root)
    l = []
    for case in cases:
        files = os.listdir(os.path.dirname(case))
        study_id = os.path.dirname(case).split('/')[-1]

        d = {}
        d['study_id'] = study_id

        for i in files:
            if i.endswith("tw2_tra.nrrd"):
                d['t2w'] = i
            elif i.endswith("adc_tra.nrrd"):
                d['adc'] = i
            elif i.endswith("seg.nrrd"):
                d['lesion_seg'] = i
            elif i.endswith("t.nrrd"):
                d['dwi'] = i

        for i in ['t2w', 'adc', 'lesion_seg', 'dwi']:
            if i not in d:
                d[i] = None

        l.append(d)
    ds = pd.DataFrame(l)
    return ds

target_dir_root = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-new-2/*/*segmentation-label.seg.nrrd"
ds = get_available_dataset(target_dir_root)

In [168]:
# let's fix missi
ds_missing = ds[ds['dwi'].isnull() | ds['adc'].isnull() | ds['t2w'].isnull()]
ds_complete = ds[~(ds['dwi'].isnull() | ds['adc'].isnull() | ds['t2w'].isnull())]

In [169]:
ds_missing['study_id'][0]

'4c177202-954661e4-997f7da5-dd28eff8-83f62076'

In [170]:
# Initialize orthanc client
orthanc_client = Orthanc('http://localhost:8042')
orthanc_client.setup_credentials('dev-user-alta', 'SyTP&8JbKFx@a6R65^sE`Z$') 

# test conntection
patient_ids = orthanc.get_patients()
len(patient_ids)


2023-12-22 13:47:04,272 - HTTP Request: GET http://localhost:8042/patients "HTTP/1.1 200 OK"


9473

In [172]:
# print the summary with SeriesDescription 

# for study_id in ds_missing['study_id'].tolist():
#     study = pyorthanc.Study(study_id, orthanc_client)
#     print(f"\n-----{study.uid}------")
#     for s in study.series:
#         main_info = s.get_main_information()
#         d = {
#             "SeriesDescription": None, 
#             "ProtocolName": None, 
#             "SequenceName": None, 
#         }
        
#         if 'SeriesDescription' in main_info['MainDicomTags']:
#             d['SeriesDescription'] = main_info['MainDicomTags']['SeriesDescription']
#         if 'ProtocolName' in main_info['MainDicomTags']:
#             d['ProtocolName'] = main_info['MainDicomTags']['ProtocolName']
#         if 'SequenceName' in main_info['MainDicomTags']:
#             d['SequenceName'] = main_info['MainDicomTags']['SequenceName']
#         print(d)

In [173]:
import monai

In [174]:
ds_complete.iloc[0]



study_id           cbd6c770-78b7c21f-37ecf9fe-ae375e64-a771c880
t2w           4ebbe8c6-0ab2deb2-c61cb0f3-d2bf90a5-2ec05be3-t...
lesion_seg    670817f7-0a924fe5-3f2dd0ed-0f23c8ae-5899c5ce-s...
adc           d1f5fdb4-c38c5c54-88652446-7a8cdd65-616b3283-a...
dwi           0aafb099-73580ca8-5facdd6e-e35fdf2a-12add031-d...
Name: 1, dtype: object

In [175]:
from monai.transforms import LoadImageD

In [193]:
ds_complete['study_id']

1      cbd6c770-78b7c21f-37ecf9fe-ae375e64-a771c880
2      0eed5870-cef82e9b-5a2d4332-42c911cc-3c3f76da
3      882fd046-1d9d9072-4668728a-eae7c9bf-ff06a1ee
4      a9d987f2-cdc76f5a-571f5119-6b91b283-794f4b8b
5      03b292be-f6809bb6-8f2f1df6-f2bc5dda-146db4d4
                           ...                     
359    b91fdd36-c7520be0-49033d50-0f52c1fe-3f208b2f
360    0fce8620-a5ca3fd3-785a3f70-19a1f357-e96e00c9
361    ae6b2ad4-d0f65c4d-a056fdf9-f8cc8d56-a73b7f9b
362    a43e7e88-b91761f4-75cc3e00-016bcb26-3efc1377
363    c4cafddf-9a75da3e-2b915130-776ccd43-72a10482
Name: study_id, Length: 340, dtype: object

In [195]:
columns_to_modify = ['t2w', 'adc', 'dwi', 'lesion_seg']

dataset_path = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-new-2"

# Append string to each value in the specified columns
# df['FilePath'] = df[file_name_column].apply(lambda x: os.path.join(base_path, x + '.txt'))

for col in columns_to_modify:
    ds_complete[col] = ds_complete.apply(lambda row: os.path.join(dataset_path, row['study_id'], row[col]), axis=1)

In [204]:
ds_complete

,study_id,t2w,lesion_seg,adc,dwi
1,cbd6c770-78b7c21f-37ecf9fe-ae375e64-a771c880,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
2,0eed5870-cef82e9b-5a2d4332-42c911cc-3c3f76da,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
3,882fd046-1d9d9072-4668728a-eae7c9bf-ff06a1ee,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
4,a9d987f2-cdc76f5a-571f5119-6b91b283-794f4b8b,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
5,03b292be-f6809bb6-8f2f1df6-f2bc5dda-146db4d4,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
...,...,...,...,...,...
359,b91fdd36-c7520be0-49033d50-0f52c1fe-3f208b2f,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
360,0fce8620-a5ca3fd3-785a3f70-19a1f357-e96e00c9,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
361,ae6b2ad4-d0f65c4d-a056fdf9-f8cc8d56-a73b7f9b,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...
362,a43e7e88-b91761f4-75cc3e00-016bcb26-3efc1377,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...,/data/oleksii/Prostate-Lesion-Datasets-NRRDS/A...


In [207]:
ds_missing = ds[ds['dwi'].isnull() | ds['adc'].isnull() | ds['t2w'].isnull()]

In [208]:
ds_missing

,study_id,t2w,lesion_seg,adc,dwi
0,4c177202-954661e4-997f7da5-dd28eff8-83f62076,30172c4e-a4b83ad0-e99a084e-900398d1-96216e9f-t...,1b923331-d44fe250-b1d65e9e-b6c5f378-ed133a12-s...,a15ddde5-de208ff3-182f5627-6212ee3d-e8177456-a...,None
16,8cfa5252-e29dca0e-9ec20a08-27ba24bb-5c3839bc,5dc88eb1-999a7f50-367f059b-d7f393f5-73512115-t...,cda86806-a273b759-d973daf0-c1dc0945-1b5e6a4f-s...,None,None
20,f8b09479-5352c7f6-0a373a53-5f5939e8-5f4905e4,747704d0-19304603-f8b006de-0a3634e2-2686ccf2-t...,d234e1e5-8e3b308a-753ab108-bd8fe941-8bc0f6fd-s...,None,None
24,e4cec960-02b904fd-ddc90c10-e5e7704a-e834b274,4bb34e5d-dca5e7d9-ccccd080-2d747d2a-5040d4b8-t...,0576a577-e65f46f0-eb30f0d8-220a284b-b80e31eb-s...,d31e374f-e5047c3d-317f69d7-14555ae1-7b7639f2-a...,None
41,5777c9c3-3bf0845f-f0ec14a2-00842305-93ec4355,8fea57ae-5ee96a21-9e191852-3fd59a0b-f48cb77f-t...,29fa42d7-4af48937-b6cff5bc-2c547c14-195b4c1e-s...,cf86eb81-c5a4b6ee-acf130a6-ddbd5e0b-981a556c-a...,None
78,91fbca0c-b7b9b939-d1712aea-6ae85dea-6e2a9465,2b03b06b-1784d40c-203eb301-e5dc6578-99ab5e17-t...,44aca9b8-6daf5ea6-e34ec972-9909318b-9d8c61ab-s...,1ecc2dc1-a8564bcf-de17f912-e0ced940-061bbeea-a...,None
107,b837c3a2-892b1bbe-0fed18b3-dfcedae0-755c3ed2,99d43cba-9cd45c3a-c59f3c39-1e1b4bcd-12868b76-t...,5295e2f8-25fb0e87-2d805bd2-ef932a10-30007005-s...,74faac4f-a0414bb8-4d8119f7-ab6445c2-b1aeeca3-a...,None
114,065481dd-cf736003-307153ef-ab7d799e-9030b454,9aa34417-861a037c-1ffe114e-4e114180-3b4bc9f2-t...,3fe40853-eadaea4d-4cc6c0fe-b62011d3-9cd6064a-s...,2b799b3a-81df3f8c-30404f24-55354066-9167e55d-a...,None
120,155d2337-d48bc74f-b2618bb7-2517b585-97dd04cc,8a3f0e6b-d1e2840b-15ab0f47-772fe0c4-0265b6a9-t...,b3d80662-16982f93-6c72a252-851d5290-394b90f8-s...,f8743a28-60e51c6e-cbedd09e-dc9c16b2-bbfd2062-a...,None
141,5ab9a54e-58155d26-dfe8542f-b91c14d3-dd1a6091,298329f2-52c0fd48-976dd347-5377f6a5-4c03c135-t...,77ff8d19-0cd6c5ff-4e0222ca-697402be-936d7d77-s...,594814a3-29995b31-1aefe91f-0979aed5-e2dddddb-a...,None


In [205]:
data = LoadImageD(keys=("t2w", "lesion_seg"))(ds_complete.iloc[0])

In [206]:
data['t2w'].shape

torch.Size([640, 640, 24])

In [ ]:
ds_complete

In [221]:
def get_alta_dataset(data_path, seg_path):
    studies = os.listdir(data_path)
    records = []
    for study_id in studies:
        
        study_path = os.path.join(data_path, study_id)
        study_path_seg = os.path.join(seg_path, study_id)
        
        t2ws = glob.glob(os.path.join(study_path, "*-tw2_tra.nrrd"))
        adcs = glob.glob(os.path.join(study_path, "*-adc_tra.nrrd"))
        dwis = glob.glob(os.path.join(study_path, "*-dwi_tra_bval_*.nrrd"))
        segs = glob.glob(os.path.join(study_path_seg, "*.seg.nrrd"))

        d = {}
        d['study_id'] = study_id
        
        d["t2w"] = t2ws[0] if t2ws else None  
        d["adc"] = adcs[0] if adcs else None  
        d["dwis"] = ','.join(dwis) if dwis else None  
        d["seg"] = segs[0] if segs else None  
        

        records.append(d)
        
    ds = pd.DataFrame(records)
    return ds


data_path = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/"
seg_path = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-seg/"
    
ds = get_alta_dataset(data_path, seg_path)
ds_missing = ds[ds['dwis'].isnull() | ds['adc'].isnull() | ds['t2w'].isnull()]
print(len(ds))

1922


In [222]:
data_path_new = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-missing/"
seg_path_new = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-seg-missing/"

In [224]:
# os.makedirs(data_path_new)
# os.makedirs(seg_path_new)
# 
# for oid in ds_missing['study_id'].tolist():
#     shutil.move(os.path.join(data_path, oid), data_path_new)
#     shutil.move(os.path.join(seg_path, oid), seg_path_new)
#     print(os.path.join(data_path, oid), os.path.join(data_path_new))
#     print(os.path.join(seg_path, oid), os.path.join(seg_path_new))
#     print('-----')
    
    
# # shutil.move()

In [249]:
# oid = "61137e48-cd18b676-4b22bd38-fbdda715-457d03cd"

# oid = 'cfff4530-845f38f7-ee6ec7f6-62158fb2-39e78895'
# oid = '34bf9b66-06ef1fb6-6bc34cb4-850148df-1ca99f7c'
# oid = '08ac8a39-1ac90fb9-05e90724-1c651e44-542a35c3'
oid = '49b8118d-77b5d4ec-61d4bad1-98e3c253-c7fc3e3c'
oid = 'af9233a9-3ee360d5-a63fd6dc-1d9d4f64-f563066f'
oid = 'ae6b2ad4-d0f65c4d-a056fdf9-f8cc8d56-a73b7f9b'
oid = 'b2c53258-91f173e3-ded7a9c0-36faac17-b9383448'

# # Traceback (most recent call last):
#   File "data_loading/lesion_datasets/preprocess_lesion_data.py", line 440, in <module>
#     prepared_data = preprocess_lesion_data(case, df_lesion)
#   File "data_loading/lesion_datasets/preprocess_lesion_data.py", line 294, in preprocess_lesion_data
#     mr_img_t2 = crop_and_padd_sitk(mr_img_t2, NEW_SIZE)
#   File "/home/oleksii/projects/DeepLearningExamples/PyTorch/Segmentation/nnUNet/utils/sitk_utils.py", line 400, in crop_and_padd_sitk
#     paddedImg = filter.Execute(croppedImg)
#   File "/home/oleksii/projects/DeepLearningExamples/PyTorch/Segmentation/nnUNet/projectenv/lib/python3.8/site-packages/SimpleITK/SimpleITK.py", line 20205, in Execute
#     return _SimpleITK.ConstantPadImageFilter_Execute(self, image1)
# RuntimeError: Exception thrown in SimpleITK ConstantPadImageFilter_Execute: /tmp/SimpleITK/Code/Common/include/sitkMemberFunctionFactory.hxx:155:
# sitk::ERROR: Pixel type: vector of 16-bit signed integer is not supported in 3D by N3itk6simple22ConstantPadImageFilterE.
oid = '59365e28-7c59cd88-a88065ba-65e55b52-ef0a7cd7'
oid = 'd12485d6-eb6fe4dd-29dc8afa-6b4ae029-3739da58'
oid = 'd8196ddc-da8f071d-d3ca8a13-e3c87b8b-3e4876ae'
oid = '3ec044bd-d2fa5e7f-3f1cdd40-842ceebc-cc0d3a7d'
shutil.move(os.path.join(data_path, oid), "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/-ALTA-Lesion-Dataset-fist-batch-lost/")
shutil.move(os.path.join(seg_path, oid), "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/-ALTA-Lesion-Dataset-fist-batch-lost-seg/")

'/data/oleksii/Prostate-Lesion-Datasets-NRRDS/-ALTA-Lesion-Dataset-fist-batch-lost-seg/3ec044bd-d2fa5e7f-3f1cdd40-842ceebc-cc0d3a7d'

In [277]:
dwi = glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/*/tra_dwi_calc_bval_*.nrrd")

In [269]:
t2ws = glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/*/tra_t2w.nrrd")

In [270]:
t2w_img = sitk.ReadImage(t2ws[0])

In [274]:
t2w_arr = sitk.GetArrayFromImage(t2w_img)


In [276]:
t2w_arr.dtype


dtype('int16')

In [267]:
dwi


['/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/d55838e1-51531d28-e9da1f29-926450d2-1bdcbe85/tra_dwi_calc_bval_1000.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/4c177202-954661e4-997f7da5-dd28eff8-83f62076/tra_dwi_calc_bval_1000.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/0ff11760-bd590db0-78fe0467-335a1905-ee6d9520/tra_dwi_calc_bval_1000.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/cbd6c770-78b7c21f-37ecf9fe-ae375e64-a771c880/tra_dwi_calc_bval_1000.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/6f675ad8-8cdb7105-565491c1-4ebbda34-184d991c/tra_dwi_calc_bval_1000.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/0eed5870-cef82e9b-5a2d4332-42c911cc-3c3f76da/tra_dwi_calc_bval_1000.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-pre

In [265]:
for i_path in dwi: 
#     os.rename(i_path, os.path.join(os.path.dirname(i_path), 'tra_dwi_calc_bval_1000.nrrd'))
    os.rename(i_path, os.path.join(os.path.dirname(i_path), 'tra_dwi_calc_bval_1000.nrrd'))   

In [280]:
for i_path in dwi:
    print(i_path)
    img = sitk.ReadImage(i_path)
    arr = sitk.GetArrayFromImage(img)
    arr = np.rint(arr)
    arr.astype(np.uint16)
    img_cast = sitk.GetImageFromArray(arr)
    img_cast.CopyInformation(img)
    img_cast = sitk.Cast(img_cast, sitk.sitkUInt16)
    sitk.WriteImage(img_cast, i_path)

/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/d55838e1-51531d28-e9da1f29-926450d2-1bdcbe85/tra_dwi_calc_bval_1000.nrrd
/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/4c177202-954661e4-997f7da5-dd28eff8-83f62076/tra_dwi_calc_bval_1000.nrrd
/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/0ff11760-bd590db0-78fe0467-335a1905-ee6d9520/tra_dwi_calc_bval_1000.nrrd
/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/cbd6c770-78b7c21f-37ecf9fe-ae375e64-a771c880/tra_dwi_calc_bval_1000.nrrd
/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/6f675ad8-8cdb7105-565491c1-4ebbda34-184d991c/tra_dwi_calc_bval_1000.nrrd
/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/0eed5870-cef82e9b-5a2d4332-42c911cc-3c3f76da/tra_dwi_calc_bval_1000.nrrd
/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-preprocessed/cca9828e-3d2e3d2